In [ ]:
!rm -rf `find -type d -name .ipynb_checkpoints`

In [1]:
import torch
from torch.utils.data import DataLoader
from PIL import Image
import json
import re
from typing import Optional
from codecarbon import EmissionsTracker
from os import listdir
from os.path import isfile, join
from tqdm.auto import tqdm
from PIL import Image
from tqdm.notebook import tqdm
from transformers import ViltConfig
from transformers import ViltProcessor
from transformers import ViltForQuestionAnswering
from carbontracker.tracker import CarbonTracker
import numpy as np
from dataloader import *
from train import *
torch.manual_seed(41)

In [2]:
config = ViltConfig.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
tr_image_root = '/home/ahuq/research/data/VQA_data/v1/images/train2014/'
tr_q_root = open('/home/ahuq/research/data/VQA_data/v1/questions/Questions_Train_mscoco/OpenEnded_mscoco_train2014_questions.json')
tr_a_root = open('/home/ahuq/research/data/VQA_data/v1/annotations/Annotations_Train_mscoco/mscoco_train2014_annotations.json')

val_image_root = '/home/ahuq/research/data/VQA_data/v1/images/val2014'
val_q_root = open('/home/ahuq/research/data/VQA_data/v1/questions/Questions_Val_mscoco/OpenEnded_mscoco_val2014_questions.json')
val_a_root = open('/home/ahuq/research/data/VQA_data/v1/annotations/Annotations_Val_mscoco/mscoco_val2014_annotations.json')

ts_image_root = '/home/ahuq/research/data/VQA_data/v1/images/val2014'
ts_q_root = open('/home/ahuq/research/data/VQA_data/v1/questions/Questions_Val_mscoco/OpenEnded_mscoco_val2014_questions.json')
ts_a_root = open('/home/ahuq/research/data/VQA_data/v1/annotations/Annotations_Val_mscoco/mscoco_val2014_annotations.json')

print("--------Training Data-------")
tr_questions, tr_annotations, tr_id_to_filename = load_data(tr_image_root,tr_q_root,tr_a_root, config)
print("--------Validation Data-------")
val_questions, val_annotations, val_id_to_filename = load_data(val_image_root,val_q_root,val_a_root, config)
print("--------Testing Data-------")
ts_questions, ts_annotations, ts_id_to_filename = load_data(ts_image_root,ts_q_root,ts_a_root, config)

--------Training Data-------


  0%|          | 0/82783 [00:00<?, ?it/s]

data questions keys: dict_keys(['info', 'task_type', 'data_type', 'license', 'data_subtype', 'questions'])
Number of questions: 248349
data annotations keys: dict_keys(['info', 'data_type', 'license', 'data_subtype', 'annotations'])
Number of annotations: 248349


  0%|          | 0/248349 [00:00<?, ?it/s]

--------Validation Data-------


...

  0%|          | 0/121512 [00:00<?, ?it/s]

--------Testing Data-------


  0%|          | 0/40504 [00:00<?, ?it/s]

data questions keys: dict_keys(['info', 'task_type', 'data_type', 'license', 'data_subtype', 'questions'])
Number of questions: 121512
data annotations keys: dict_keys(['info', 'data_type', 'license', 'data_subtype', 'annotations'])
Number of annotations: 121512


  0%|          | 0/121512 [00:00<?, ?it/s]

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-mlm")

tr_size = int(len(tr_questions)*0.1)
val_size = int(len(val_questions)*0.1)

tr_dataset = VQADataset(questions = tr_questions[:tr_size], 
                        annotations = tr_annotations[:tr_size], 
                        id_to_filename = tr_id_to_filename, 
                        processor = processor, 
                        config = config)
val_dataset = VQADataset(questions = val_questions[:val_size], 
                         annotations = val_annotations[:val_size], 
                         id_to_filename = val_id_to_filename, 
                         processor = processor,
                         config = config)
ts_dataset = VQADataset(questions = val_questions[100000:100000+1000], 
                        annotations = val_annotations[100000:100000+1000], 
                        id_to_filename = val_id_to_filename, 
                        processor = processor, 
                        config = config)


model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm",id2label=config.id2label,label2id=config.label2id)
model = model.to(device)

Some weights of ViltForQuestionAnswering were not initialized from the model checkpoint at dandelin/vilt-b32-mlm and are newly initialized: ['classifier.0.bias', 'classifier.0.weight', 'classifier.1.bias', 'classifier.1.weight', 'classifier.3.bias', 'classifier.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
print("--------Training Data-------")

train_dataloader = DataLoader(tr_dataset, collate_fn=collate_fn, batch_size=32, shuffle=True)
tr_batch = next(iter(train_dataloader))
for k,v in tr_batch.items():
    print(k, v.shape)
    
print("--------Validation Data-------")
val_dataloader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=32, shuffle=True)
val_batch = next(iter(val_dataloader))
for k,v in val_batch.items():
    print(k, v.shape)

print("--------Testing Data-------")
test_dataloader = DataLoader(ts_dataset, collate_fn=collate_fn, batch_size=4, shuffle=True)
ts_batch = next(iter(test_dataloader))
for k,v in ts_batch.items():
    print(k, v.shape)

--------Training Data-------
input_ids torch.Size([32, 40])
attention_mask torch.Size([32, 40])
token_type_ids torch.Size([32, 40])
pixel_values torch.Size([32, 3, 576, 608])
pixel_mask torch.Size([32, 576, 608])
labels torch.Size([32, 3129])
--------Validation Data-------
input_ids torch.Size([32, 40])
attention_mask torch.Size([32, 40])
token_type_ids torch.Size([32, 40])
pixel_values torch.Size([32, 3, 576, 608])
pixel_mask torch.Size([32, 576, 608])
labels torch.Size([32, 3129])
--------Testing Data-------
input_ids torch.Size([4, 40])
attention_mask torch.Size([4, 40])
token_type_ids torch.Size([4, 40])
pixel_values torch.Size([4, 3, 384, 576])
pixel_mask torch.Size([4, 384, 576])
labels torch.Size([4, 3129])


In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

# Optimizer setup
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
epochs = 100
model_name = 'teacher_0.3'
save_path = '/home/ahuq/research/CS791_LLM_Cleaned/models/'
tracker = EmissionsTracker()
tracker.start_task("teacher_0.3")
teacher_train_results = train_and_validate(model,train_dataloader, val_dataloader, optimizer, epochs,save_path, model_name, device)
step_emissions = tracker.stop_task("teacher_0.3")
# Stop tracking emissions for the task
emissions = tracker.stop()
print(f"Emissions for the step: {step_emissions} kg CO2")
print(f"Emissions for the task: {emissions} kg CO2")

In [5]:
#student model

sample = ViltConfig.from_pretrained("dandelin/vilt-b32-mlm",id2label=config.id2label, label2id=config.label2id)
sample.num_hidden_layers = 6  # Reduce number of transformer layers (e.g., half the original)
sample.hidden_size = 384   #768    # Reduce the hidden size (e.g., smaller than original 768)
student_model = ViltForQuestionAnswering(sample)
student_model = student_model.to(device)
# print(student_model)

In [6]:
total_params = sum(p.numel() for p in student_model.parameters())
print("Total parameters:", total_params)

# Get the number of trainable parameters
trainable_params = sum(p.numel() for p in student_model.parameters() if p.requires_grad)
print("Trainable parameters:", trainable_params)

Total parameters: 33560121
Trainable parameters: 33560121


In [ ]:
optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)
epochs = 100
model_name = 'student_few'
save_path = '/home/ahuq/research/CS791_LLM_Cleaned/student_models/'
tracker = EmissionsTracker()
tracker.start_task("student_0.3")
student_train_results = train_and_validate(student_model,train_dataloader, val_dataloader, optimizer, epochs,save_path, model_name, device)
step_emissions = tracker.stop_task("student_0.3")
# Stop tracking emissions for the task
emissions = tracker.stop()
print(f"Emissions for the step: {step_emissions} kg CO2")
print(f"Emissions for the task: {emissions} kg CO2")

In [7]:
val_batch = next(iter(val_dataloader))
val_batch = {k: v.to(device) for k, v in val_batch.items()}

teacher_model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm",id2label=config.id2label,label2id=config.label2id)  # Initialize with any required arguments
teacher_path = '/home/ahuq/research/CS791_LLM_Cleaned/models/teacher_0.3.pth'
teacher_model.load_state_dict(torch.load(teacher_path))

student_model = ViltForQuestionAnswering(sample)
student_model = student_model.to(device)
student_path = '/home/ahuq/research/CS791_LLM_Cleaned/student_models/student_few.pth'
student_model.load_state_dict(torch.load(student_path))

student_model= student_model.to(device)
teacher_model = teacher_model.to(device)

student_model.eval()
teacher_model.eval()

teacher_output = test(teacher_model, test_dataloader, device)
student_output = test(student_model, test_dataloader, device)
# with torch.no_grad():
#     student_outputs = student_model(**val_batch, output_hidden_states=True, output_attentions=True)
    # teacher_outputs = teacher_model(**val_batch, output_hidden_states=True, output_attentions=True)

Some weights of ViltForQuestionAnswering were not initialized from the model checkpoint at dandelin/vilt-b32-mlm and are newly initialized: ['classifier.0.bias', 'classifier.0.weight', 'classifier.1.bias', 'classifier.1.weight', 'classifier.3.bias', 'classifier.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_235138/2778914859.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless 

Validation - Loss: 7.2856, Top-5 Accuracy: 0.5895, Overall Accuracy: 0.3028


Validation: 100%|█████████████████████████████████████████████████████████████████████| 250/250 [02:15<00:00,  1.84it/s]

Validation - Loss: 9.9244, Top-5 Accuracy: 0.4824, Overall Accuracy: 0.2414


In [8]:
teacher_output = test(teacher_model, val_dataloader, device)
student_output = test(student_model, val_dataloader, device)

Validation: 100%|█████████████████████████████████████████████████████████████████████| 380/380 [05:58<00:00,  1.06it/s]


Validation - Loss: 6.9447, Top-5 Accuracy: 0.6076, Overall Accuracy: 0.3130


Validation: 100%|█████████████████████████████████████████████████████████████████████| 380/380 [05:40<00:00,  1.12it/s]

Validation - Loss: 9.5433, Top-5 Accuracy: 0.5063, Overall Accuracy: 0.2458


In [9]:
teacher_output = test(teacher_model, train_dataloader, device)
student_output = test(student_model, train_dataloader, device)

Validation: 100%|█████████████████████████████████████████████████████████████████████| 777/777 [12:20<00:00,  1.05it/s]


Validation - Loss: 0.6246, Top-5 Accuracy: 0.9848, Overall Accuracy: 0.5131


Validation: 100%|█████████████████████████████████████████████████████████████████████| 777/777 [11:33<00:00,  1.12it/s]

Validation - Loss: 0.6572, Top-5 Accuracy: 0.9847, Overall Accuracy: 0.5134


In [11]:
def get_results(model):
    student_output = test(model, train_dataloader, device)
    student_output = test(model, val_dataloader, device)
    student_output = test(model, test_dataloader, device)

student_model = ViltForQuestionAnswering(sample)
student_model = student_model.to(device)
student_path = '/home/ahuq/research/CS791_LLM_Cleaned/student_models/student_logit.pth'
student_model.load_state_dict(torch.load(student_path))

get_results(student_model)

# student_model = ViltForQuestionAnswering(sample)
# student_model = student_model.to(device)
# student_path = '/home/ahuq/research/CS791_LLM_Cleaned/student_models/few/student_hidden.pth'
# student_model.load_state_dict(torch.load(student_path))

# get_results(student_model)

# student_model = ViltForQuestionAnswering(sample)
# student_model = student_model.to(device)
# student_path = '/home/ahuq/research/CS791_LLM_Cleaned/student_models/few/student_attention.pth'
# student_model.load_state_dict(torch.load(student_path))

# get_results(student_model)

# student_model = ViltForQuestionAnswering(sample)
# student_model = student_model.to(device)
# student_path = '/home/ahuq/research/CS791_LLM_Cleaned/student_models/few/student_hybrid.pth'
# student_model.load_state_dict(torch.load(student_path))

# get_results(student_model)

/tmp/ipykernel_235138/1823811148.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student_model.load_state_dict(torch.load(student_path))
Validation: 100%|███████████████

Validation - Loss: 0.6341, Top-5 Accuracy: 0.9849, Overall Accuracy: 0.5131


Validation: 100%|█████████████████████████████████████████████████████████████████████| 380/380 [05:36<00:00,  1.13it/s]


Validation - Loss: 9.3543, Top-5 Accuracy: 0.5323, Overall Accuracy: 0.2482


Validation: 100%|█████████████████████████████████████████████████████████████████████| 250/250 [02:09<00:00,  1.93it/s]

Validation - Loss: 9.5773, Top-5 Accuracy: 0.5131, Overall Accuracy: 0.2510


In [ ]:
### logits based train student model
sample = ViltConfig.from_pretrained("dandelin/vilt-b32-mlm",
                                                 id2label=config.id2label,
                                                 label2id=config.label2id)
sample.num_hidden_layers = 6  # Reduce number of transformer layers (e.g., half the original)
sample.hidden_size = 384   #768    # Reduce the hidden size (e.g., smaller than original 768)
student_model = ViltForQuestionAnswering(sample)

teacher_model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm",id2label=config.id2label,label2id=config.label2id)  # Initialize with any required arguments
teacher_path = '/home/ahuq/research/CS791_LLM_Cleaned/models/teacher_0.3.pth'
teacher_model.load_state_dict(torch.load(teacher_path))

student_model= student_model.to(device)
teacher_model = teacher_model.to(device)
student_model.eval()
teacher_model.eval()

epochs = 100
alpha = 0.7
save_path = '/home/ahuq/research/CS791_LLM_Cleaned/student_models/'
optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)
tracker = EmissionsTracker()
tracker.start_task("student_logits_0.3")
KD_logit_results = KD_train_based_logits(teacher_model, student_model, train_dataloader, alpha, optimizer, epochs,save_path,device)
step_emissions = tracker.stop_task("student_logits_0.3")
# Stop tracking emissions for the task
emissions = tracker.stop()
print(f"Emissions for the step: {step_emissions} kg CO2")
print(f"Emissions for the task: {emissions} kg CO2")

In [ ]:
### hidden states KD student training 
sample = ViltConfig.from_pretrained("dandelin/vilt-b32-mlm",
                                                 id2label=config.id2label,
                                                 label2id=config.label2id)
sample.num_hidden_layers = 6  # Reduce number of transformer layers (e.g., half the original)
sample.hidden_size = 384   #768    # Reduce the hidden size (e.g., smaller than original 768)
student_model = ViltForQuestionAnswering(sample)

teacher_model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm",id2label=config.id2label,label2id=config.label2id)  # Initialize with any required arguments
teacher_path = '/home/ahuq/research/CS791_LLM_Cleaned/models/teacher_0.3.pth'
teacher_model.load_state_dict(torch.load(teacher_path))

student_model= student_model.to(device)
teacher_model = teacher_model.to(device)
# student_model.eval()
teacher_model.eval()

epochs = 100
alpha = 0.7
save_path = '/home/ahuq/research/CS791_LLM_Cleaned/student_models/'
# Instantiate the projector (e.g., teacher 768 -> student 384)
projector = HiddenStateProjector(teacher_dim=768, student_dim=384).to('cpu')
optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)
tracker = EmissionsTracker()
tracker.start_task("student_hidden_0.3")
KD_hidden_results = KD_train_based_hidden_states(teacher_model, student_model, train_dataloader, projector, alpha, optimizer, epochs,save_path,device)
step_emissions = tracker.stop_task("student_hidden_0.3")
# Stop tracking emissions for the task
emissions = tracker.stop()
print(f"Emissions for the step: {step_emissions} kg CO2")
print(f"Emissions for the task: {emissions} kg CO2")

In [ ]:
#student training attention based

sample = ViltConfig.from_pretrained("dandelin/vilt-b32-mlm",
                                                 id2label=config.id2label,
                                                 label2id=config.label2id)
sample.num_hidden_layers = 6  # Reduce number of transformer layers (e.g., half the original)
sample.hidden_size = 384   #768    # Reduce the hidden size (e.g., smaller than original 768)
student_model = ViltForQuestionAnswering(sample)

teacher_model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm",id2label=config.id2label,label2id=config.label2id)  # Initialize with any required arguments
teacher_path = '/home/ahuq/research/CS791_LLM_Cleaned/models/teacher_0.3.pth'
teacher_model.load_state_dict(torch.load(teacher_path))

student_model= student_model.to(device)
teacher_model = teacher_model.to(device)
student_model.eval()
teacher_model.eval()

epochs = 100
alpha = 0.7
mapping_strategy = 'direct'
save_path = '/home/ahuq/research/CS791_LLM_Cleaned/student_models/'
optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)
tracker = EmissionsTracker()
tracker.start_task("student_attention_0.3")
KD_attention_results = KD_train_based_attentions(teacher_model, student_model, train_dataloader, projector, alpha, mapping_strategy, optimizer, epochs,save_path, device)
step_emissions = tracker.stop_task("student_attention_0.3")
# Stop tracking emissions for the task
emissions = tracker.stop()
print(f"Emissions for the step: {step_emissions} kg CO2")
print(f"Emissions for the task: {emissions} kg CO2")

In [ ]:
#Hybrid KD
sample = ViltConfig.from_pretrained("dandelin/vilt-b32-mlm",
                                                 id2label=config.id2label,
                                                 label2id=config.label2id)
sample.num_hidden_layers = 6  # Reduce number of transformer layers (e.g., half the original)
sample.hidden_size = 384   #768    # Reduce the hidden size (e.g., smaller than original 768)
student_model = ViltForQuestionAnswering(sample)

teacher_model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm",id2label=config.id2label,label2id=config.label2id)  # Initialize with any required arguments
teacher_path = '/home/ahuq/research/CS791_LLM_Cleaned/models/teacher_0.3.pth'
teacher_model.load_state_dict(torch.load(teacher_path))

student_model= student_model.to(device)
teacher_model = teacher_model.to(device)
student_model.eval()
teacher_model.eval()

epochs = 100
alpha = 0.7
mapping_strategy = 'direct'
save_path = '/home/ahuq/research/CS791_LLM_Cleaned/student_models/'
optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)
projector = HiddenStateProjector(teacher_dim=768, student_dim=384).to('cpu')

tracker = EmissionsTracker()
tracker.start_task("student_hybrid_0.3")
# Your training code here

KD_hybrid_results = KD_train_based_hybrid(teacher_model, student_model, train_dataloader, projector, alpha, mapping_strategy, optimizer, epochs, save_path, device)
step_emissions = tracker.stop_task("student_hybrid_0.3")
# Stop tracking emissions for the task
emissions = tracker.stop()
print(f"Emissions for the step: {step_emissions} kg CO2")
print(f"Emissions for the task: {emissions} kg CO2")

In [ ]:
print(emissions)

In [ ]:
from codecarbon import EmissionsTracker

In [ ]:
from carbontracker.tracker import CarbonTracker

In [ ]:
!pip install --upgrade codecarbon

In [ ]:
from codecarbon import EmissionsTracker

# Initialize the emissions tracker
tracker = EmissionsTracker()

# Start tracking emissions for a task
tracker.start()

# Simulate a task
tracker.start_task("test")
for i in range(1000000):
    pass 
step_emissions = tracker.stop_task("test")

# Stop tracking emissions for the task
emissions = tracker.stop()

# Print the emissions data
print(f"Emissions for the step: {step_emissions} kg CO2")
print(f"Emissions for the task: {emissions} kg CO2")